In [3]:
import numpy as np
import cv2

from ffpyplayer.player import MediaPlayer

In [4]:
def play_video(path):
    frame_name = 'Video'
    vid = cv2.VideoCapture(path)
    audio = MediaPlayer(path)
    
    cv2.namedWindow(frame_name, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(frame_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while True:
        ret, frame = vid.read()
        audio_frame, val = audio.get_frame()
        if not ret:
            print('End of video...')
            break
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break
        cv2.imshow(frame_name, frame)
        if val != 'eof' and audio_frame is not None:
            img, t = audio_frame
        
    vid.release()

In [5]:
play_video('data/izone_fiesta_mcountdown.webm')